In [1]:
# Enable autoreload of imported files
%load_ext autoreload
%autoreload 2

In [2]:
# Create workspace and experiment objects and start a run
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129193")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129193
Azure region: southcentralus
Subscription id: b19c9c81-5f59-4537-b3c0-c1beb163ec22
Resource group: aml-quickstarts-129193


In [3]:
# Create compute cluster or use existing one
from azureml.core.compute import ComputeTarget, AmlCompute

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Copy the training script to the training directory (which will be the entry directory for the SKLearn estimator)
import os
import shutil

if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', "./training")

'./training/train.py'

In [5]:
# Prepare hyperdrive run
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import loguniform, choice

# Specify parameter sampler
ps = RandomParameterSampling({"--C": loguniform(-10, 10),
                              "--max_iter": choice(1, 10, 100, 1000, 10000)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=1)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./training",
              compute_target=compute_target,
              entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_e987d538-1334-421a-8a0c-f7c0c4a40d66
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e987d538-1334-421a-8a0c-f7c0c4a40d66?wsid=/subscriptions/b19c9c81-5f59-4537-b3c0-c1beb163ec22/resourcegroups/aml-quickstarts-129193/workspaces/quick-starts-ws-129193

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-04T20:17:53.566981][API][INFO]Experiment created<END>\n""<START>[2020-12-04T20:17:54.045402][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-04T20:17:54.222203][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-04T20:17:54.7226903Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
# Get the best run and save the model from that run.
import joblib

best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_hyperdrive_run.get_details()['runDefinition']['arguments'])

In [ ]:
best_hyperdrive_run.download_file()

In [ ]:
print(best_run.get_file_names())

In [ ]:
# Register best model
model = best_run.register_model(model_name='udacity-project-model', model_path='outputs/model.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
training_df = clean_data(ds, split=False)

In [ ]:
# Write df to datastore and retrieve it as dataset object
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(training_df).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)
 
# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
label = "y"

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name=label,
    compute_target=compute_target,
    n_cross_validations=5)

In [ ]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output = False)

In [ ]:
# Retrieve and save your best automl model.
automl_run.wait_for_completion()
best_run, model = automl_run.get_output()

In [ ]:
import joblib
joblib.dump(model, 'outputs/automl_model.pkl')